# Clipper Tutorial: Part 2

In this part of the tutorial, you will put on your data scientist hat and train and deploy some models to Clipper to improve your application accuracy.


# Connect to Clipper (again)

Because this is a separate Python instance, you must create a new `Clipper` object and connect to your running Clipper instance. Make sure you enter the same information here as you did in part one.

In [ ]:
# clipper_manager must be on your path:
import sys
import os
sys.path.append(os.path.abspath('../../management/'))
import clipper_manager as cm
# Change the username if necessary
user = ""
# Set the path to the SSH key
key = ""
# Set the SSH host
host = ""
clipper = cm.Clipper(host, user, key)

# Load Cifar

Because this is a new notebook, you must load the CIFAR dataset again. This time, you will be using it to train and evaluate machine learning models.

*Set the location to the directory where you downloaded the CIFAR dataset in part one.*

In [ ]:
cifar_loc = ""
import cifar_utils
train_x, train_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_train.data", norm=True))
test_x, test_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=True))

# Train Logistic Regression Model

When tackling a new problem with machine learning, it's always good to start with simple models and only add complexity when needed. Start by training a logistic regression binary classifier using [Scikit-Learn](http://scikit-learn.org/). This model gets about 74% accuracy on the offline evaluation dataset.

In [ ]:
from sklearn import linear_model as lm 
def train_sklearn_model(m, train_x, train_y):
    m.fit(train_x, train_y)
    return m
lr_model = train_sklearn_model(lm.LogisticRegression(), train_x, train_y)
print("Logistic Regression test score: %f" % lr_model.score(test_x, test_y))

# Deploy Logistic Regression Model

While 74% accuracy on a CIFAR binary classification task is significantly below state of the art, it's already much better than the 50% accuracy of random guessing that you're getting right now without machine learning.

You can deploy your logistic regression model directly to Clipper without having to worry about how to serialize the model or integrate it with application code.

To deploy a model to Clipper, you must assign it a name ("sklearn_cifar"), a version (1), and then provide some metadata about the model itself. In this case, you are specifying that you want to run the model using the `sklearn_cifar_container` Docker image in the [Clipper repo](https://hub.docker.com/u/clipper/dashboard/) on Docker Hub. You can assign the model descriptive labels, and specify the input type that this model expects. Finally, you can specify how many *replicas* of the model (how many Docker containers) to launch. Adding more replicas increases the throughput of this model.

> *Once again, because you are deploying a Docker image this command may take awhile to download the image. Thanks for being patient!*

In [ ]:
model_added = clipper.deploy_model(
    "sklearn_cifar",
    1,
    lr_model,
    "clipper/sklearn_cifar_container:latest",
    ["cifar", "sklearn"],
    "doubles",
    num_containers=1
)
print("Model deploy successful? {success}".format(success=model_added))

Now that you've deployed your model, go ahead and check back on your running frontend application from part 1. You should see the accuracy rise from around 50% to around 74%, without having to stop or modify your application at all!

# Load TensorFlow Model

To improve the accuracy of your application further, you will now deploy a TensorFlow convolutional neural network. This model takes a few hours to train, so you will download the trained model parameters rather than training it from scratch. This model gets about 88% accuracy on the test dataset.

There is a pre-trained TensorFlow model stored in the repo using [`git-lfs`](https://git-lfs.github.com/). Once you install `git-lfs`, you can download the model with the command `git lfs pull`. If you don't want to deploy a TensorFlow model, you can skip this step.

In [ ]:
import os
import tensorflow as tf
import numpy as np
tf_cifar_model_path = os.path.abspath("tf_cifar_model/cifar10_model_full")
tf_session = tf.Session('', tf.Graph())
with tf_session.graph.as_default():
    saver = tf.train.import_meta_graph("%s.meta" % tf_cifar_model_path)
    saver.restore(tf_session, tf_cifar_model_path)

def tensorflow_score(session, test_x, test_y):
    """
    NOTE: This predict method expects pre-whitened (normalized) images
    """
    logits = session.run('softmax_logits:0',
                           feed_dict={'x:0': test_x})
    relevant_activations = logits[:, [cifar_utils.negative_class, cifar_utils.positive_class]]
    preds = np.argmax(relevant_activations, axis=1)
    return float(np.sum(preds == test_y)) / float(len(test_y))
print("TensorFlow CNN test score: %f" % tensorflow_score(tf_session, test_x, test_y))

# Deploy TensorFlow Model

Similar to deploying the logistic regression model, you can now deploy your TensorFlow neural network. Note that you will specify a different model container to use this time: the `tf_cifar_container`.

> *Once again, please patient while the Docker image is downloaded.*

In [ ]:
model_added = clipper.deploy_model(
    "tf_cifar",
    1,
    os.path.abspath("tf_cifar_model"),
    "clipper/tf_cifar_container:latest",
    ["cifar", "tf"],
    "doubles",
    num_containers=1
)
print("Model deploy successful? {success}".format(success=model_added))

# Inspect Selection Policy

When you register an application with multiple candidate models, the selection policy dictates which of those models to query, and if more than one model is queried, how to combine the results. Selection policies can be used to implement ensembles, to perform dynamic online model selection using multi-armed bandit algorithms (a more statistically efficient alternative to A/B testing), to estimate prediction confidence, and many other applications. Selection policies can learn from feedback and be updated in real-time directly in the serving system. For more information about selection policies in Clipper, check out the [research paper on Arxiv](https://arxiv.org/abs/1612.03079).

When you created the `cifar_demo` application, you registered it to use the EXP4 selection policy. This selection policy always uses all of the candidate models, and will average the predictions together. When provided with feedback, EXP4 will learn a weighted average that gives more weight to models that are correct more often.

You have note provided Clipper with any feedback yet, so EXP4 gives your two models equal weight.

You can see this by inspecting the selection policy weights for user 0 (the default user).

In [ ]:
app_name = "cifar_demo"
clipper.inspect_selection_policy(app_name, 0)

If you check out the accuracy of your frontend application again in the notebook from part 1, you'll see that it should be around 80% accuracy, about halfway between 74% and 88% accuracy.

# Start feedback

To have EXP4 learn to give more weight to the more accurate TensorFlow neural network, enable feedback for your application. This will send 100 pieces of feedback (input images labeled with the correct label) to Clipper through the feedback REST api.

In [ ]:
cifar_utils.enable_feedback(host, app_name, test_x, test_y, 100)

# Inspect Selection Policy Again

Now, when you inspect the EXP4 weights again, you should see that Clipper has learned to place more weight on the more accurate TensorFlow model and less weight on the simple logistic regression model.

In [ ]:
clipper.inspect_selection_policy(app_name, 0)

If you check the accuracy of your frontend application a final time, you should see accuracy around 88%. If the accuracy is below that, you can try sending more feedback to increase the weight on the TensorFlow model even more.

# Inspect Clipper Metrics

Clipper also records various system performance metrics. You can inspect the current state of these metrics with the `inspect_instance()` command.

In [ ]:
clipper.inspect_instance()

__Congratulations! You've now successfully completed the tutorial. You started Clipper, created an application and queried it from a frontend client, and deployed two models trained in two different machine learning frameworks (Scikit-Learn and TensorFlow) to the running system.__

*Head back to the notebook from part 1. When you're done watching the accuracy of your application, stop the cell (hit the little "stop" square in the notebook toolbar).*